In [83]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.static import teams
from json import JSONDecodeError
import pandas as pd
import numpy as np
import random
import requests
import math
import time

In [93]:
def str_to_mins(inp):
    lst = inp.split(":")
    sec = int(lst[0])*60 + int(lst[1])
    return sec

In [94]:
dataset = pd.read_csv("../mixed-model/training_data_mixed_model_v2.csv")
dataset = dataset.dropna(axis = 0)

In [95]:
dataset.shape

(19170, 463)

In [96]:
#convert to final numpy form
import numpy as np 

# Labels are the values we want to predict
labels = np.array(dataset['result'])
# Remove the labels column; axis 1 refers to the columns
values = dataset.drop(['result'], axis = 1)
# Saving feature names for later use
feature_list = list(values.columns)
# Convert to numpy array
values = np.array(values)

In [97]:
#split into training, validation, and testing data
from sklearn.model_selection import train_test_split

X = values
y = labels

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=1) #this gives 20% test, 20% val, 60% train

In [100]:
#here's the magical step! We train the classifier
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 500 decision trees
rf = RandomForestClassifier(n_estimators = 200, random_state = 42, verbose = 1, max_features = 0.5)
# Train the model on training data
rf.fit(X_train, y_train);

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  6.0min finished


In [101]:
# the next part of the code is to predict based on the validation set and evaluate the performance of the model...
# we don't want to use the test set until we're certain that we're absolutely done with tweaking parameters
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y_pred = rf.predict(X_val)
# Model Accuracy, how often is the classifier correct?
print("Accuracy: ",metrics.accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
confusion_matrix(y_val, y_pred, labels = [0, 1])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy:  0.726395409494001
              precision    recall  f1-score   support

           0       0.67      0.60      0.63      1499
           1       0.76      0.81      0.78      2335

    accuracy                           0.73      3834
   macro avg       0.71      0.70      0.71      3834
weighted avg       0.72      0.73      0.72      3834



[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.1s finished


array([[ 904,  595],
       [ 454, 1881]], dtype=int64)

In [98]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

X_scaled = preprocessing.scale(X_train)

lr = LogisticRegression(random_state=0, verbose = 1, max_iter=100000)
lr.fit(X_scaled, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s finished


LogisticRegression(max_iter=100000, random_state=0, verbose=1)

In [99]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn import metrics

y_pred = lr.predict(preprocessing.scale(X_val))
# Model Accuracy, how often is the classifier correct?
print("Accuracy: ",metrics.accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

Accuracy:  0.7167449139280125
              precision    recall  f1-score   support

           0       0.65      0.59      0.62      1499
           1       0.75      0.80      0.77      2335

    accuracy                           0.72      3834
   macro avg       0.70      0.69      0.70      3834
weighted avg       0.71      0.72      0.71      3834



In [81]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [82]:
# the next part of the code is to predict based on the validation set and evaluate the performance of the model...
# we don't want to use the test set until we're certain that we're absolutely done with tweaking parameters
from sklearn import metrics
from sklearn.metrics import classification_report

y_pred = clf.predict(X_val)
# Model Accuracy, how often is the classifier correct?
print("Accuracy: ",metrics.accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

Accuracy:  0.7339593114241002
              precision    recall  f1-score   support

           0       0.68      0.61      0.64      1499
           1       0.76      0.81      0.79      2335

    accuracy                           0.73      3834
   macro avg       0.72      0.71      0.71      3834
weighted avg       0.73      0.73      0.73      3834

